In [2]:
import requests as rq
import pandas as pd
import json
from bs4 import BeautifulSoup
import re

# Getting data papers from Scientific Data

In [2]:
ApiEndpoint = 'https://api.openalex.org/works?filter=primary_location.source.id:'
ScientificDataId = 'S2607323502'

## Get all SD publications and filter for data papers type

In [4]:
# Init values of the loop
page = 1
finalResults = []
has_more_pages = True
fewer_than_10k_results = True
# Init Scientific Data URL
init_url = ApiEndpoint + ScientificDataId + '&page={}'

# Pagination: loop through pages
while has_more_pages and fewer_than_10k_results:
    # Set page value and request page from OpenAlex
    url = init_url.format(page)
    print('\n' + url)
    page_with_results = rq.get(url).json()
    results = page_with_results['results']
    for i,work in enumerate(results):
        ## Check if it is a data descriptor (this would change depends on the journal)
        try:
            sourcePage = rq.get(work['doi'])
            soup = BeautifulSoup(sourcePage.content, "html.parser")
            script_tag = soup.find("script")
            if script_tag:
                script_tag_contents = script_tag.string
                if 'data descriptor' in script_tag_contents:
                    print("yes it is a data descriptor")
                    finalResults.append(work)
        except:
            print("error requesting DOI")
    # next page
    page += 1
    
    # end loop when either there are no more results on the requested page 
    # or the next request would exceed 10,000 results
    per_page = page_with_results['meta']['per_page']
    has_more_pages = len(results) == per_page
    fewer_than_10k_results = per_page * page <= 10000
    #%store finalResults
    #%store page


https://api.openalex.org/works?filter=primary_location.source.id:S2607323502&page=1
The FAIR Guiding Principles for scientific data management and stewardship
MIMIC-III, a freely accessible critical care database
yes it is a data descriptor
The climate hazards infrared precipitation with stations—a new environmental record for monitoring extremes
yes it is a data descriptor
Present and future Köppen-Geiger climate classification maps at 1-km resolution
yes it is a data descriptor
Climatologies at high resolution for the earth’s land surface areas
yes it is a data descriptor
Version 4 of the CRU TS monthly high-resolution gridded multivariate climate dataset
yes it is a data descriptor
Advancing The Cancer Genome Atlas glioma MRI collections with expert segmentation labels and radiomic features
yes it is a data descriptor
The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions
yes it is a data descriptor
TerraClimate, a high-resolu

## Save the list of data papers in SD (if needed)

In [ ]:
import csv
# With all the filtered results we get the final formatted excel
keys = finalResults[0].keys()
with open('resultsSD.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(finalResults)

In [ ]:
%store df1

## Fixing missing PDF reference in some data papers

In [6]:
%store -r df1
counter = 0
## You need to load the previous created list in df1 varaible
for index, row in df1.iterrows():
 
    if (row['primary_location.pdf_url'] == None):
        #headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1'}
        page = rq.get(row['primary_location.landing_page_url'], allow_redirects=True)
        soup = BeautifulSoup(page.content, "html.parser")
        typeer = soup.find_all("meta")
        type = soup.find("meta", attrs={"name":"citation_pdf_url"})
        typeContent = type.attrs["content"]
        df1.at[index,'primary_location.pdf_url'] = typeContent

df1.to_csv('resultsPDF.csv', index=False)

## Get the PDF of the data papers and save it SData/documents

This process may take long time.

In [9]:
for index, row in df1.iterrows():
    if (row['primary_location.pdf_url'] != None):
        url = row['primary_location.pdf_url']
        id = row['primary_location.pdf_url'].rsplit('/', 1)[-1]
        r = rq.get(url, stream=True)
        with open('SData/documents/'+str(id), 'wb') as f:
            f.write(r.content)

Parse all the PDF attach metadata to the CSV

# Getting data papers from Data-in-Brief

In [ ]:
import requests as rq

ApiEndpoint = 'https://api.openalex.org/works?filter=primary_location.source.id:'
dataBriefId = 'S2481029507'


## Get all DBrief publications and filter for data papers type

In [42]:
# Init values of the loop
page = 1
dataBriefResults = []
has_more_pages = True
fewer_than_10k_results = True

# Init Scientific Data URL
init_url = ApiEndpoint + dataBriefId + '&page={}'

# Pagination: loop through pages
while has_more_pages and fewer_than_10k_results:
    
    # Set page value and request page from OpenAlex
    url = init_url.format(page)
    print('\n' + url)
    page_with_results = rq.get(url).json()
    
    results = page_with_results['results']

    for i,work in enumerate(results):
        print(work['title'])
        result = rq.head(work['doi'], allow_redirects=True)
        id = result.url.rsplit('/', 1)[-1]
        headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1'}
        sourcePage = rq.get('https://www.sciencedirect.com/science/article/pii/'+id, headers=headers)
        soup = BeautifulSoup(sourcePage.content, "html.parser")
        type = soup.find("meta", attrs={"name":"citation_article_type"})
        typeContent = type.attrs["content"]
        if "Data article" in typeContent:
            print("yes it is a data descriptor")
            dataBriefResults.append(work)
        #dataBriefResults.append(work)
    # next page
    page += 1
    
    # end loop when either there are no more results on the requested page 
    # or the next request would exceed 10,000 results
    per_page = page_with_results['meta']['per_page']
    has_more_pages = len(results) == per_page
    fewer_than_10k_results = per_page * page <= 10000


https://api.openalex.org/works?filter=primary_location.source.id:S2481029507&page=1
Application of the ARIMA model on the COVID-2019 epidemic dataset
Dataset of breast ultrasound images
Chorological maps for the main European woody species
Data supporting the spectrophotometric method for the estimation of catalase activity
Considerations and code for partial volume correcting [ 18 F]-AV-1451 tau PET data
Database on the mechanical properties of high entropy alloys and complex concentrated alloys
SDNET2018: An annotated image dataset for non-contact concrete crack detection using deep convolutional neural networks
Experimental data on the properties of natural fiber particle reinforced polymer composite material
Bearing vibration data collected under time-varying rotational speed conditions
A citrus fruits and leaves dataset for detection and classification of citrus diseases through machine learning
Impacts of the Covid-19 Pandemic on Life of Higher Education Students: Global Survey 

In [52]:
# Cleaning and sampling DBrief data papers
dataBriefResults2 = []
for results in dataBriefResults:

    results.pop('abstract_inverted_index')
    dataBriefResults2.append(results)

dataResults = pd.json_normalize(dataBriefResults2)
# With all the filtered results we get the final formatted excel
dfDBrief = pd.DataFrame.from_dict(dataResults)

# Sampling, frac value may change, we initally used 0.25
dfBrief2 = dfDBrief.sample(frac=0.011)
dfBrief2.to_csv("RemainigDBriefPapers.csv")

## Getting PDFs of Data-In-Brief

This process may take a long time

In [7]:
remaining = pd.read_csv('DBrief/RemainigDBriefPapers.csv')

for index, data_paper in remaining.iterrows():
    if index > 7003:
        if isinstance(data_paper['doi'], str):
            doi = data_paper['doi'].replace('https://doi.org/', '')
            ScienceDirectEndpoint="https://api.elsevier.com/content/article/doi/"+doi+"?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464"
            print(ScienceDirectEndpoint)
            response = rq.get(ScienceDirectEndpoint, stream=True)
            with open('./DBrief/documents/'+doi.rsplit('/', 1)[-1]+'.pdf', 'wb') as f:
                f.write(response.content)

https://api.elsevier.com/content/article/doi/10.1016/j.dib.2016.12.039?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/article/doi/10.1016/j.dib.2017.01.001?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/article/doi/10.1016/j.dib.2017.02.012?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/article/doi/10.1016/j.dib.2017.02.010?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/article/doi/10.1016/j.dib.2017.02.018?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/article/doi/10.1016/j.dib.2017.02.016?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/article/doi/10.1016/j.dib.2017.02.039?httpAccept=application/pdf&apiKey=50088199f1d0aa2cc6875ad8f30e1464
https://api.elsevier.com/content/a